# Import

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [4]:
train_dataset,target=load_script.load_dataset(train=True)

In [5]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [6]:
single_target=(target[:,1]>target[:,0]).type(torch.FloatTensor)
single_test_target=(test_target[:,1]>test_target[:,0]).type(torch.FloatTensor)

In [7]:
train_dataset[0]

Variable containing:
  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [8]:
target[0]

Variable containing:
 1
 0
[torch.FloatTensor of size 2]

# Defining the network

### Simplest network as possible

In [5]:
F.max_pool1d(kernel_size=3)

TypeError: max_pool1d() missing 1 required positional argument: 'input'

In [9]:
class Simple(nn.Module):
    def __init__(self):
        super(Simple,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,2)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

### Got from this paper https://tdk.bme.hu/VIK/DownloadPaper/Kezmozdulatok-detektalasa-EEG-jel-alapjan

In [10]:
class Paper(nn.Module):
    def __init__(self):
        super(Paper,self).__init__()

        self.conv1=nn.Conv1d(28,28,5)
        self.full1=nn.Linear(28*23,512)
        self.full2=nn.Linear(512,512)
        self.full3=nn.Linear(512,2)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),2)

        x=F.relu(self.full1(x.view(-1,28*23)))
        x=F.relu(self.full2(x))
        
        x=F.sigmoid(self.full3(x))
        
        return x

### Single output

In [11]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

# Training

### Standard training

In [12]:
net=Paper()

optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion=nn.MSELoss()
mini_batch_size=1

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        
#         print(train_element.shape)
        out=net(train_element)
#         print(out)
        output_target[b:b+mini_batch_size]=(out[:,1]>out[:,0]).data
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
#         print(out)
    output_test=(out[:,1]>out[:,0]).data.type(torch.FloatTensor)
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target==target[:,1].data))/target.shape[0]
    error_test=np.sum(list(output_test==test_target[:,1].data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 89.37661999091506 Correct: 54.74% Correct test: 49.0%
Epoch: 10 Loss: 69.01822441164404 Correct: 67.40% Correct test: 51.0%
Epoch: 20 Loss: 61.20998738706112 Correct: 73.41% Correct test: 51.0%
Epoch: 30 Loss: 54.76791105698794 Correct: 76.58% Correct test: 52.0%
Epoch: 40 Loss: 48.00250121485442 Correct: 81.01% Correct test: 53.0%
Epoch: 50 Loss: 41.47877614246681 Correct: 85.12% Correct test: 56.0%
Epoch: 60 Loss: 35.262712138239294 Correct: 88.60% Correct test: 56.0%
Epoch: 70 Loss: 29.459269742888864 Correct: 91.45% Correct test: 60.0%
Epoch: 80 Loss: 24.260416189405078 Correct: 93.03% Correct test: 61.0%
Epoch: 90 Loss: 19.640424454242748 Correct: 96.20% Correct test: 64.0%
Epoch: 100 Loss: 15.789122350417529 Correct: 98.73% Correct test: 66.0%
Epoch: 110 Loss: 12.59495668621912 Correct: 98.73% Correct test: 65.0%
Epoch: 120 Loss: 10.074616905423227 Correct: 98.73% Correct test: 67.0%


KeyboardInterrupt: 

### Single output training

In [ ]:
net=SingleOutput()

optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion=nn.MSELoss()
mini_batch_size=1

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(single_target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=single_target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        
#         print(train_element.shape)
        out=net(train_element)
#         print(out)
        output_target[b:b+mini_batch_size]=(out>0.5).data
#         print(out)
#         print(target_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(single_test_target.shape[0])
    out=net(test_dataset)
#         print(out)
    output_test=(out>0.5).data.type(torch.FloatTensor)
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target==single_target.data))/single_target.shape[0]
    error_test=np.sum(list(output_test[:,0]==single_test_target.data))/single_test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")